In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils import data

import scipy.io as scio
import numpy as np

## Initialize Hyper-parameters

In [2]:
input_size = 11 # low dimension resolution
hidden_size = 100 #
num_classes = 10
num_epochs = 30
batch_size = 10
learning_rate = 0.001

## Feedforward Neural Network Model Structure

In [3]:
class ShallowNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(ShallowNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self,x):
        out = self.fc1(x)
        out = self.relu(out)
        out =self.fc2(out)
        return out

In [4]:
# Creating TrainLoader
# load low_dim training data
low_dim_data = scio.loadmat('low_dim_6000.mat')
low_dim = low_dim_data['low_dim'].reshape(6000)
low_dim = np.array([i.reshape(11) for i in low_dim])
low_dim -= low_dim.min()
low_dim /= low_dim.max()
# load labels
orig_data = scio.loadmat('subset_6000.mat')
labels = orig_data['subset_label_X']
labels = labels.reshape(6000)
# convert to dataLoader

tensor_low_dim = torch.Tensor(low_dim[:5000,:])
tensor_labels = torch.Tensor(labels[:5000])
low_dim_dataset = data.TensorDataset(tensor_low_dim, tensor_labels)
train_loader = data.DataLoader(dataset=low_dim_dataset,
                              batch_size=batch_size,)
print(tensor_low_dim.shape)

torch.Size([5000, 11])


In [5]:
# Creating TestLoader
# load_low_dim test data
low_dim_data = scio.loadmat('low_dim_test_1000.mat')
low_dim = low_dim_data['low_dim_test'].reshape(1000)
low_dim = np.array([i.reshape(11) for i in low_dim])
low_dim -= low_dim.min()
low_dim /= low_dim.max()
# load labels
labels = low_dim_data['subset_label_X_test']
labels = labels.reshape(1000)
# # convert to dataLoader
tensor_low_dim = torch.Tensor(low_dim[:,:])
tensor_labels = torch.Tensor(labels[:])
low_dim_dataset = data.TensorDataset(tensor_low_dim, tensor_labels)
test_loader = data.DataLoader(dataset=low_dim_dataset,
                              batch_size=batch_size,)
print(tensor_low_dim.shape)

torch.Size([1000, 11])


## Instantiate the NN

In [6]:
net = ShallowNetwork(input_size, hidden_size, num_classes)

## Enable GPU 

In [53]:
# net.cuda()    # You can comment out this line to disable GPU

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

## Train the Model

In [8]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        images = Variable(images)         
        labels = Variable(labels.long())
        
        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        outputs = net(images)                           # Forward pass: compute the output class given a image
        loss = criterion(outputs, labels)                 # Compute the loss: difference between the output class and the pre-given label
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes
        if (i+1) % 100 == 0:                              # Logging
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, 6000//batch_size, loss.data))

Epoch [1/30], Step [100/600], Loss: 2.2701
Epoch [1/30], Step [200/600], Loss: 2.1618
Epoch [1/30], Step [300/600], Loss: 2.0353
Epoch [1/30], Step [400/600], Loss: 1.8263
Epoch [1/30], Step [500/600], Loss: 1.7833
Epoch [2/30], Step [100/600], Loss: 1.8775
Epoch [2/30], Step [200/600], Loss: 1.6330
Epoch [2/30], Step [300/600], Loss: 1.4896
Epoch [2/30], Step [400/600], Loss: 1.3363
Epoch [2/30], Step [500/600], Loss: 1.2551
Epoch [3/30], Step [100/600], Loss: 1.5787
Epoch [3/30], Step [200/600], Loss: 1.2345
Epoch [3/30], Step [300/600], Loss: 1.1714
Epoch [3/30], Step [400/600], Loss: 1.0686
Epoch [3/30], Step [500/600], Loss: 0.9734
Epoch [4/30], Step [100/600], Loss: 1.4148
Epoch [4/30], Step [200/600], Loss: 1.0454
Epoch [4/30], Step [300/600], Loss: 1.0410
Epoch [4/30], Step [400/600], Loss: 0.9189
Epoch [4/30], Step [500/600], Loss: 0.8339
Epoch [5/30], Step [100/600], Loss: 1.3088
Epoch [5/30], Step [200/600], Loss: 0.9561
Epoch [5/30], Step [300/600], Loss: 0.9795
Epoch [5/30

## Test the Model

Similar to training the nerual network, we also need to load batches of test images and collect the outputs. The differences are that:
(1) No loss & weights calculation
(2) No wights update
(3) Has correct prediction calculation

In [9]:
correct = 0
total = 0
for images, labels in test_loader:
    labels = labels.long()
    images = Variable(images)
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)  # Choose the best class from the output: The class with the best score
    total += labels.size(0)                    # Increment the total count
    correct += (predicted == labels).sum()     # Increment the correct count
    
print('Accuracy of the network on the 10K test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10K test images: 17 %


## Save the trained Model for future use

In [11]:
# Save the model
torch.save(net.state_dict(), 'fnn_model.pkl')

In [10]:
from tensorboardX import SummaryWriter
writer = SummaryWriter('runs/mnist')

In [11]:
imgs = None
for images, labels in test_loader:
    imgs = images
    break
writer.add_graph(net,tensor_low_dim)

In [127]:
writer.add_graph(net,tensor_low_dim)

In [1]:
writer.close()

NameError: name 'writer' is not defined